In [68]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding,Concatenate,Input,Bidirectional,LSTM,Dense,Dropout,GRU
from sklearn.model_selection import train_test_split

In [41]:
data = pd.read_csv('DATASET.csv')
print(data.head())
print(data.shape)

                                              Review     label
0  Great music service, the audio is high quality...  POSITIVE
1  Please ignore previous negative rating. This a...  POSITIVE
2  This pop-up "Get the best Spotify experience o...  NEGATIVE
3    Really buggy and terrible to use as of recently  NEGATIVE
4  Dear Spotify why do I get songs that I didn't ...  NEGATIVE
(52702, 2)


In [42]:
data.isna().sum()

Review    16
label      0
dtype: int64

In [43]:
data = data.dropna()
data.shape

(52686, 2)

In [53]:
data['Review'] = data['Review'].astype(str).str.replace(r'Ã[\x80-\xBF]+|[^a-zA-Z\s]|\s+', ' ', regex=True).str.strip().str.lower()

In [54]:
tokenized_reviews = []
for review in data['Review']:
    tokens = word_tokenize(review)
    tokenized_reviews.append(tokens)
data['Review Tokenized'] = tokenized_reviews
data.head()

,Review,label,Tokenized Text,Text Without Stopwords,Text Lemmatized,Review Tokenized
0,great music service the audio is high quality ...,POSITIVE,"[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ...","[great, music, service, the, audio, is, high, ..."
1,please ignore previous negative rating this ap...,POSITIVE,"[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,...","[please, ignore, previous, negative, rating, t..."
2,this pop up get the best spotify experience on...,NEGATIVE,"[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,...","[this, pop, up, get, the, best, spotify, exper..."
3,really buggy and terrible to use as of recently,NEGATIVE,"[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]","[really, buggy, and, terrible, to, use, as, of..."
4,dear spotify why do i get songs that i didn t ...,NEGATIVE,"[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, why, do, i, get, songs, that, ..."


In [55]:
stop_words = stopwords.words('english')
reviews_no_stopwords = []
for review in data['Review Tokenized']:
    review_no_stopwords = [word for word in review if word not in stop_words]
    reviews_no_stopwords.append(review_no_stopwords)
data['Review Without Stopwords'] = reviews_no_stopwords
data.head()

,Review,label,Tokenized Text,Text Without Stopwords,Text Lemmatized,Review Tokenized,Review Without Stopwords
0,great music service the audio is high quality ...,POSITIVE,"[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ...","[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ..."
1,please ignore previous negative rating this ap...,POSITIVE,"[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,...","[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a..."
2,this pop up get the best spotify experience on...,NEGATIVE,"[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,...","[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,..."
3,really buggy and terrible to use as of recently,NEGATIVE,"[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]","[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]"
4,dear spotify why do i get songs that i didn t ...,NEGATIVE,"[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu..."


In [56]:
lemmatizer = WordNetLemmatizer()
lemmatized_reviews = []
for review in data['Review Without Stopwords']:
    lemmatized_review = [lemmatizer.lemmatize(word,pos='v') for word in review]
    lemmatized_reviews.append(lemmatized_review)
data['Review Lemmatized'] = lemmatized_reviews
data.head()

,Review,label,Tokenized Text,Text Without Stopwords,Text Lemmatized,Review Tokenized,Review Without Stopwords,Review Lemmatized
0,great music service the audio is high quality ...,POSITIVE,"[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ...","[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ..."
1,please ignore previous negative rating this ap...,POSITIVE,"[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,...","[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,..."
2,this pop up get the best spotify experience on...,NEGATIVE,"[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,...","[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,..."
3,really buggy and terrible to use as of recently,NEGATIVE,"[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]","[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]"
4,dear spotify why do i get songs that i didn t ...,NEGATIVE,"[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu..."


In [61]:
processed_reviews = []
for review in data['Review Lemmatized']:
    review = ' '.join(review)
    processed_reviews.append(review)
data['Processed Reviews'] = processed_reviews
data.head()

,Review,label,Tokenized Text,Text Without Stopwords,Text Lemmatized,Review Tokenized,Review Without Stopwords,Review Lemmatized,Processed Review,Processed Reviews
0,great music service the audio is high quality ...,POSITIVE,"[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ...","[great, music, service, the, audio, is, high, ...","[great, music, service, audio, high, quality, ...","[great, music, service, audio, high, quality, ...",great music service audio high quality app eas...,great music service audio high quality app eas...
1,please ignore previous negative rating this ap...,POSITIVE,"[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,...","[please, ignore, previous, negative, rating, t...","[please, ignore, previous, negative, rating, a...","[please, ignore, previous, negative, rat, app,...",please ignore previous negative rat app super ...,please ignore previous negative rat app super ...
2,this pop up get the best spotify experience on...,NEGATIVE,"[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,...","[this, pop, up, get, the, best, spotify, exper...","[pop, get, best, spotify, experience, android,...","[pop, get, best, spotify, experience, android,...",pop get best spotify experience android annoy ...,pop get best spotify experience android annoy ...
3,really buggy and terrible to use as of recently,NEGATIVE,"[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]","[really, buggy, and, terrible, to, use, as, of...","[really, buggy, terrible, use, recently]","[really, buggy, terrible, use, recently]",really buggy terrible use recently,really buggy terrible use recently
4,dear spotify why do i get songs that i didn t ...,NEGATIVE,"[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, why, do, i, get, songs, that, ...","[dear, spotify, get, songs, put, playlist, shu...","[dear, spotify, get, songs, put, playlist, shu...",dear spotify get songs put playlist shuffle play,dear spotify get songs put playlist shuffle play


In [62]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Processed Reviews'])
vocab_size = len(tokenizer.word_index) + 1
vocab_size

17454

In [63]:
input_sequences = tokenizer.texts_to_sequences(data['Review'])
input_sequences = np.array(pad_sequences(input_sequences, padding='pre'))

In [64]:
y = data['label'].map({'POSITIVE':1, 'NEGATIVE':0})
y

0        1
1        1
2        0
3        0
4        0
        ..
52697    1
52698    1
52699    1
52700    1
52701    1
Name: label, Length: 52686, dtype: int64

In [65]:
x_train, x_test, y_train, y_test = train_test_split(input_sequences, y, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25)

In [112]:
x_train.shape

(31611, 131)

In [119]:
model = Sequential([
    Embedding(vocab_size,131),
    Bidirectional(LSTM(131, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_18 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_61                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_62                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [120]:
early_stopping = EarlyStopping(monitor='val_accuracy',mode='max',patience=5,verbose=1, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val,y_val), verbose=1, callbacks=[early_stopping])

Epoch 1/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 220s 220ms/step - accuracy: 0.8120 - loss: 0.4042 - val_accuracy: 0.8708 - val_loss: 0.3097
Epoch 2/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 221s 224ms/step - accuracy: 0.8901 - loss: 0.2637 - val_accuracy: 0.8660 - val_loss: 0.3228
Epoch 3/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 221s 224ms/step - accuracy: 0.9109 - loss: 0.2255 - val_accuracy: 0.8634 - val_loss: 0.3404
Epoch 4/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 223s 225ms/step - accuracy: 0.9207 - loss: 0.1897 - val_accuracy: 0.8613 - val_loss: 0.3548
Epoch 5/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 222s 225ms/step - accuracy: 0.9325 - loss: 0.1656 - val_accuracy: 0.8573 - val_loss: 0.3988
Epoch 6/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 218s 220ms/step - accuracy: 0.9385 - loss: 0.1442 - val_accuracy: 0.8545 - val_loss: 0.4207
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [121]:
pred = model.predict(x_test)
y_pred = [(1 if p>=0.5 else 0) for p in pred]
y_test = list(y_test)
matching = [(1 if y_pred[i]==y_test[i] else 0) for i in range(len(y_test))]
test_accuracy = np.sum(matching)/len(matching)
print('Test Accuracy:',test_accuracy)

330/330 ━━━━━━━━━━━━━━━━━━━━ 16s 49ms/step
Test Accuracy: 0.8673372556462327


In [129]:
pred_label = [('POSITIVE' if p==1 else 'NEGATIVE') for p in y_pred]
submission = pd.DataFrame({'label':pred_label})
submission.to_csv('submission.csv',index_label='id')